In [42]:
from bs4 import BeautifulSoup   
from selenium import webdriver  

#use the 'incognito' mode to scrape the page. completely optional 
option = webdriver.ChromeOptions()
option.add_argument("--incognito")

#need to download chromedriver first. get it here: https://chromedriver.storage.googleapis.com/index.html?path=2.40/
#below is the path to where the chromedriver.exe is. 
#you can put it somewhere else, just remember to change the path here
browser = webdriver.Chrome("D:/chromedriver_win32/chromedriver.exe", chrome_options=option)

browser.get("http://worldweather.wmo.int/tc/city.html?cityId=237")

soup = BeautifulSoup(browser.page_source ,"html.parser")  

In [49]:
#取得濕度
for i in soup.select(".present_rh_value"):
  print(i.text)

47%


In [56]:
#get the dates and temperatures
for i in soup.select(".city_fc_date.fs0"):   #兩個class
    print(i.text) 

for j in soup.select(".min_temp_box.fs0"):   #兩個class
  print(j.text)

for f in soup.select(".max_temp_icon.fs0"):
 print(f.text)

6月29日(五)
6月30日(六)
7月1日(日)
7月2日(一)
7月3日(二)
7月4日(三)
24°C
25°C
23°C
23°C
25°C
24°C
37°C
36°C
35°C
35°C
33°C
35°C
